In [ ]:
import os
import re
import git
import sys
import glob
import json
import joblib
import sklearn
import skimage
import tifffile
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import pathlib

import matplotlib
from matplotlib import pyplot as plt

In [ ]:
import dragonfly_automation.utils
from dragonfly_automation.fov_models import PipelineFOVScorer

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
fov_scorer = PipelineFOVScorer(save_dir='../models/2019-10-08/', mode='prediction')
fov_scorer.load()
fov_scorer.train()
fov_scorer.validate()

In [ ]:
from dragonfly_automation.gateway import mock_gateway

In [ ]:
gate = mock_gateway.Gate('random-real')

In [ ]:
gate.getLastMeta()

In [ ]:
# PML0216 had progressively less clumpiness, with some too-few and no-nuclei FOVs
pml_dir = pathlib.Path('/Volumes/ml_group/raw-pipeline-microscopy/PML0216/')

# PML0316 was a very good full-plate
# pml_dir = pathlib.Path('/Volumes/ml_group/raw-pipeline-microscopy/PML0316/')

In [ ]:
snaps_dir = pml_dir / 'logs/fov-scoring/fov-images/'

df = pd.read_csv(pml_dir / 'logs/fov-scoring/fov-score-log.csv')

scored = df.loc[df.score.notna()]
too_few = df.loc[df.comment == 'Too few nuclei in the FOV']
no_nuclei = df.loc[df.comment == 'No nuclei in the FOV']

scored = scored.sort_values(by='score', ascending=False)

In [ ]:
scored.shape, too_few.shape, no_nuclei.shape

In [ ]:
row = too_few.iloc[0]
filepath = snaps_dir / f'FOV_{row.position_name}_RAW.tif'
im = imageio.imread(filepath)
plt.imshow(im)

result = fov_scorer.score_raw_fov(im, min_otsu_thresh=700, min_num_nuclei=10)
row.score, result.get('score'), result.get('comment')

In [ ]:
dst_dirpath = '/Users/keith.cheveralls/projects/dragonfly-automation/dragonfly_automation/tests/artifacts/snaps/'
import shutil
shutil.copy2(filepath, os.path.join(dst_dirpath, 'too-few-1.tif'))